# Basic Usage - Simple Backtest Framework

This notebook demonstrates the basic usage of the simple-backtest library using real market data from Yahoo Finance.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

from simple_backtest import Backtest, BacktestConfig
from simple_backtest.strategy.buy_and_hold import BuyAndHoldStrategy
from simple_backtest.strategy.moving_average import MovingAverageStrategy
from simple_backtest.visualization.plotter import plot_equity_curve

## 1. Download Market Data

Let's download Apple (AAPL) stock data from Yahoo Finance.

In [ ]:
# Download data
ticker = "AAPL"
start_date = "2020-01-01"
end_date = "2023-12-31"

data = yf.download(ticker, start=start_date, end=end_date)
print(f"Downloaded {len(data)} rows of data")
print(f"Date range: {data.index[0]} to {data.index[-1]}")
data.head()

## 2. Configure Backtest

Set up the backtest configuration with initial capital, lookback period, and commission.

In [ ]:
config = BacktestConfig(
    initial_capital=10000.0,
    lookback_period=50,
    commission_type="percentage",
    commission_value=0.001,  # 0.1% commission
    execution_price="open",
    parallel_execution=True,
    risk_free_rate=0.02,  # 2% annual risk-free rate
)

print(config)

## 3. Buy and Hold Strategy

The simplest strategy - buy at the start and hold until the end.

In [ ]:
# Create buy and hold strategy
buy_hold = BuyAndHoldStrategy(shares=100)

# Run backtest
backtest = Backtest(data=data, config=config)
results = backtest.run([buy_hold])

# Display metrics
print("\n=== Buy and Hold Strategy Results ===")
for key, value in results[buy_hold.get_name()]['metrics'].items():
    print(f"{key:25s}: {value:>12.2f}")

In [ ]:
# Plot equity curve
fig = plot_equity_curve(results)
fig.show()

## 4. Moving Average Crossover Strategy

Buy when short MA crosses above long MA, sell when it crosses below.

In [ ]:
# Create moving average strategy
ma_strategy = MovingAverageStrategy(
    short_window=10,
    long_window=30,
    shares=100
)

# Run backtest
backtest = Backtest(data=data, config=config)
results = backtest.run([ma_strategy])

# Display metrics
print("\n=== Moving Average Strategy Results ===")
for key, value in results[ma_strategy.get_name()]['metrics'].items():
    print(f"{key:25s}: {value:>12.2f}")

In [ ]:
# Plot equity curve
fig = plot_equity_curve(results)
fig.show()

## 5. Compare Multiple Strategies

Let's compare different moving average configurations.

In [ ]:
# Create multiple strategies
strategies = [
    BuyAndHoldStrategy(shares=100),
    MovingAverageStrategy(short_window=5, long_window=20, shares=100),
    MovingAverageStrategy(short_window=10, long_window=30, shares=100),
    MovingAverageStrategy(short_window=20, long_window=50, shares=100),
]

# Run all strategies in parallel
backtest = Backtest(data=data, config=config)
results = backtest.run(strategies)

# Compare metrics
print("\n=== Strategy Comparison ===")
comparison_df = pd.DataFrame({
    name: res['metrics'] 
    for name, res in results.items() 
    if name != 'benchmark'
}).T

comparison_df[['total_return', 'sharpe_ratio', 'max_drawdown', 'win_rate', 'total_trades']]

In [ ]:
# Plot all equity curves
fig = plot_equity_curve(results)
fig.show()

## 6. Analyze Trade History

Let's look at the trade history for one of the strategies.

In [ ]:
# Get trade history
strategy_name = "MA_10_30"
trades = results[strategy_name]['trade_history']

# Convert to DataFrame
trades_df = pd.DataFrame(trades)
trades_df = trades_df[['timestamp', 'signal', 'shares', 'price', 'commission', 'pnl', 'cash']]

print(f"\nTotal trades: {len(trades_df)}")
trades_df.head(10)

In [ ]:
# Analyze profitable vs losing trades
sell_trades = trades_df[trades_df['signal'] == 'sell']
profitable = sell_trades[sell_trades['pnl'] > 0]
losing = sell_trades[sell_trades['pnl'] <= 0]

print(f"Profitable trades: {len(profitable)} ({len(profitable)/len(sell_trades)*100:.1f}%)")
print(f"Losing trades: {len(losing)} ({len(losing)/len(sell_trades)*100:.1f}%)")
print(f"Average profit: ${profitable['pnl'].mean():.2f}")
print(f"Average loss: ${losing['pnl'].mean():.2f}")